In [9]:
import polars as pl

import mlflow

from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import make_pipeline

import uuid


In [22]:
# We can parametrise our input and output files.
# The output file stores the 'actual_duration' vs 'predicted_duration'
year = 2021
month = 2
taxi_type = 'green'

input_file  = f'./data/{taxi_type}_tripdata_{year:04d}-{month:02d}.parquet'
output_file = f'./output/{taxi_type}_{year:04d}-{month:02d}.parquet'

RUN_ID = '068bda10a3ed4b73a771df771161f60a' 

In [23]:
# Create unique IDs for the DataFrame rows
def generate_uuids(n):
    ride_ids = []
    for i in range(n):
        ride_ids.append(str(uuid.uuid4()))
    return ride_ids

In [24]:
def read_dataframe(filename: str) -> pl.DataFrame:
    df = pl.read_parquet(filename)
    df = df.with_columns(
                (pl.col('lpep_dropoff_datetime') -
                 pl.col('lpep_pickup_datetime')).alias('duration'))
    df = df.with_columns([
           (pl.col('duration').dt.seconds() / 60)])
    df = df.filter(
            pl.any((pl.col('duration') >= 1) &
                   (pl.col('duration') <= 60)))
    categorical = ['PULocationID','DOLocationID']
    df = df.with_columns(pl.col(categorical).cast(pl.Utf8))
    # Add unique IDs as new column
    series_uuids = pl.Series(name="ride_id", 
                             values=generate_uuids(len(df)))
    df = df.with_columns(series_uuids)
    return df

In [25]:
def prepare_dictionaries(df: pl.DataFrame) ->  dict[str,float|str]:
    df= df.with_columns(pl.concat_str(
                        ['PULocationID','DOLocationID'],
                        separator="_").alias('PU_DO'))
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    dicts = df.select(pl.col(categorical+numerical)).to_dicts()
    return dicts

In [26]:
# Load the model obtained in 4.3
def load_model(run_id):
    logged_model = f'runs:/{run_id}/model'            
    model = mlflow.pyfunc.load_model(logged_model)
    return model

In [27]:
def apply_model(input_file, run_id, output_file):
    
    mlflow.set_tracking_uri("http://127.0.0.1:5000") 
    
    df = read_dataframe(input_file)
    dicts = prepare_dictionaries(df)

    
    model = load_model(run_id)
    y_pred = model.predict(dicts)

    df_result = pl.DataFrame()
    df_result = df_result.with_columns(df['ride_id'])
    df_result = df_result.with_columns(df['lpep_pickup_datetime'])
    df_result = df_result.with_columns(df['PULocationID'])
    df_result = df_result.with_columns(df['DOLocationID'])
    # Column for the duration given in input file 
    df_result = df_result.with_columns(df['duration'].alias('actual_duration'))
    # Column for predicted duration
    y_pred = pl.Series(name='predicted_duration', values=y_pred)
    df_result = df_result.with_columns(y_pred)
    # Column with difference between actual and prediction
    df_result = df_result.with_columns(
                (pl.col('actual_duration') -
                 pl.col('predicted_duration')).alias('diff'))
    # Same model for every row
    df_result = df_result.with_columns(
        pl.lit(run_id, dtype=pl.Utf8).alias("model_version")
    )
    # Write output to parquet file
    df_result.write_parquet(output_file)

In [28]:
apply_model(input_file=input_file, 
            run_id=RUN_ID,
            output_file=output_file)

2023/06/24 21:06:02 WARNING mlflow.pyfunc: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - mlflow (current: 2.3.2, required: mlflow==2.3)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.
